In [104]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import pandas as pd
from math import sqrt

In [4]:
df = pd.read_csv('/Users/kagenlim/Documents/GitHub/MA-Thesis/preprocessedstudy1.csv')

In [5]:
df.head()

,total_incidentslog1p,percentlowgradeslog1p,percentacadnotimptlog1p,school_size,urban,crime_live_recode,crime_area_recode,school_type,visitor_badge,control_building,...,privatehelp_drug,relighelp_drug,prevention_curriculum,sel,behav_mod,mentoring,peer_mediation,student_court,restorative_circles,promote_community
0,2.397895,2.772589,3.433987,3,1,1,1,Middle_Combined,1,1,...,0,0,1,1,1,1,0,0,0,1
1,2.833213,1.791759,4.394449,2,0,1,1,Middle_Combined,1,1,...,0,0,1,0,1,1,0,0,0,0
2,2.564949,1.791759,1.791759,3,1,1,1,Middle_Combined,1,1,...,1,1,1,1,1,1,1,0,0,1
3,3.258097,2.397895,2.772589,4,1,1,1,High,1,1,...,1,1,1,1,1,1,1,0,1,1
4,1.098612,1.386294,1.098612,3,1,1,1,Middle_Combined,1,1,...,0,0,1,0,1,1,0,0,0,0


In [8]:
le = LabelEncoder()
df.school_type = le.fit_transform(df.school_type)

In [9]:
df.head()

,total_incidentslog1p,percentlowgradeslog1p,percentacadnotimptlog1p,school_size,urban,crime_live_recode,crime_area_recode,school_type,visitor_badge,control_building,...,privatehelp_drug,relighelp_drug,prevention_curriculum,sel,behav_mod,mentoring,peer_mediation,student_court,restorative_circles,promote_community
0,2.397895,2.772589,3.433987,3,1,1,1,2,1,1,...,0,0,1,1,1,1,0,0,0,1
1,2.833213,1.791759,4.394449,2,0,1,1,2,1,1,...,0,0,1,0,1,1,0,0,0,0
2,2.564949,1.791759,1.791759,3,1,1,1,2,1,1,...,1,1,1,1,1,1,1,0,0,1
3,3.258097,2.397895,2.772589,4,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,1
4,1.098612,1.386294,1.098612,3,1,1,1,2,1,1,...,0,0,1,0,1,1,0,0,0,0


# Fitting Model and Evaluation

In [21]:
y = df.total_incidentslog1p

X = df.iloc[:, 1:49]

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state=42)

In [37]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [38]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor(max_features = 'sqrt', random_state=42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END bootstrap=False, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=2000; total time=   7.0s
[CV] END bootstrap=False, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=2000; total time=   6.4s
[CV] END bootstrap=False, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=2000; total time=   6.5s
[CV] END bootstrap=False, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=2000; total time=   6.4s
[CV] END bootstrap=False, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=2000; total time=   6.3s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=2, min_samples_split=2, n_estimators=1800; total time=   3.7s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=2, min_samples_split=2, n_estimators=1800; total time=   3.7s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=2, min_samples_split=2, n_estimators=180

[CV] END bootstrap=False, max_depth=10, min_samples_leaf=4, min_samples_split=10, n_estimators=1000; total time=   1.7s
[CV] END bootstrap=True, max_depth=70, min_samples_leaf=1, min_samples_split=10, n_estimators=1600; total time=   2.8s
[CV] END bootstrap=True, max_depth=70, min_samples_leaf=1, min_samples_split=10, n_estimators=1600; total time=   2.8s
[CV] END bootstrap=True, max_depth=70, min_samples_leaf=1, min_samples_split=10, n_estimators=1600; total time=   2.8s
[CV] END bootstrap=True, max_depth=70, min_samples_leaf=1, min_samples_split=10, n_estimators=1600; total time=   2.7s
[CV] END bootstrap=True, max_depth=70, min_samples_leaf=1, min_samples_split=10, n_estimators=1600; total time=   2.7s
[CV] END bootstrap=True, max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   2.3s
[CV] END bootstrap=True, max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   2.5s
[CV] END bootstrap=True, max_depth=20, min_sample

[CV] END bootstrap=True, max_depth=30, min_samples_leaf=4, min_samples_split=5, n_estimators=400; total time=   0.7s
[CV] END bootstrap=True, max_depth=30, min_samples_leaf=4, min_samples_split=5, n_estimators=400; total time=   0.7s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.4s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.4s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.4s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.4s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.4s
[CV] END bootstrap=True, max_depth=110, min_samples_leaf=4, min_samples_split=2, n_estimators=1200; total time=   2.0s
[CV] END bootstrap=True, max_depth=110, min_samples_leaf=

[CV] END bootstrap=False, max_depth=50, min_samples_leaf=1, min_samples_split=10, n_estimators=2000; total time=   4.2s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=1, min_samples_split=10, n_estimators=2000; total time=   4.2s
[CV] END bootstrap=False, max_depth=100, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   2.9s
[CV] END bootstrap=False, max_depth=100, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   2.8s
[CV] END bootstrap=False, max_depth=100, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   2.8s
[CV] END bootstrap=False, max_depth=100, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   2.8s
[CV] END bootstrap=False, max_depth=100, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   2.8s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.5s
[CV] END bootstrap=True, max_depth=80, min_

[CV] END bootstrap=False, max_depth=60, min_samples_leaf=4, min_samples_split=10, n_estimators=1400; total time=   3.4s
[CV] END bootstrap=False, max_depth=60, min_samples_leaf=4, min_samples_split=10, n_estimators=1400; total time=   3.3s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   4.6s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   4.9s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   4.8s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   4.9s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   5.4s
[CV] END bootstrap=True, max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=400; total time=   1.5s
[CV] END bootstrap=True, max_depth=50, min_sampl

[CV] END bootstrap=True, max_depth=100, min_samples_leaf=1, min_samples_split=2, n_estimators=2000; total time=   6.2s
[CV] END bootstrap=True, max_depth=100, min_samples_leaf=1, min_samples_split=2, n_estimators=2000; total time=   6.6s
[CV] END bootstrap=True, max_depth=60, min_samples_leaf=4, min_samples_split=5, n_estimators=1600; total time=   3.6s
[CV] END bootstrap=True, max_depth=60, min_samples_leaf=4, min_samples_split=5, n_estimators=1600; total time=   3.8s
[CV] END bootstrap=True, max_depth=60, min_samples_leaf=4, min_samples_split=5, n_estimators=1600; total time=   3.4s
[CV] END bootstrap=True, max_depth=60, min_samples_leaf=4, min_samples_split=5, n_estimators=1600; total time=   3.4s
[CV] END bootstrap=True, max_depth=60, min_samples_leaf=4, min_samples_split=5, n_estimators=1600; total time=   3.4s
[CV] END bootstrap=False, max_depth=70, min_samples_leaf=4, min_samples_split=10, n_estimators=800; total time=   1.9s
[CV] END bootstrap=False, max_depth=70, min_samples_l

[CV] END bootstrap=True, max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.3s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=1000; total time=   2.0s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=1000; total time=   2.1s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=1000; total time=   2.0s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=1000; total time=   2.0s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=1000; total time=   2.0s
[CV] END bootstrap=True, max_depth=90, min_samples_leaf=1, min_samples_split=2, n_estimators=1600; total time=   3.9s
[CV] END bootstrap=True, max_depth=90, min_sampl

[CV] END bootstrap=True, max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=1600; total time=   4.4s
[CV] END bootstrap=True, max_depth=50, min_samples_leaf=1, min_samples_split=2, n_estimators=1600; total time=   4.2s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.4s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.4s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.4s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.4s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.4s
[CV] END bootstrap=False, max_depth=110, min_samples_leaf=4, min_samples_split=10, n_estimators=1000; total time=   1.9s
[CV] END bootstrap=False, max_depth=110, min_samples_

RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(max_features='sqrt',
                                                   random_state=42),
                   n_iter=100,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [39]:
rf_random.best_params_

{'n_estimators': 800,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_depth': 30,
 'bootstrap': False}

In [42]:
param_gridcv = {
    'bootstrap': [False],
    'max_depth': [30, 40, 50],
    'min_samples_leaf': [2, 3, 4],
    'min_samples_split': [2, 3, 4],
    'n_estimators': [700, 800, 900, 1000]
}

In [43]:
rf = RandomForestRegressor(max_features = 'sqrt', random_state=42) #same as above#

grid = GridSearchCV(estimator = rf, param_grid = param_gridcv, 
                          cv = 5, verbose = 2)

In [44]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=700; total time=   1.7s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=700; total time=   1.7s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=700; total time=   1.6s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=700; total time=   1.6s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=700; total time=   1.7s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=800; total time=   1.9s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=800; total time=   1.9s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=800; total time= 

[CV] END bootstrap=False, max_depth=30, min_samples_leaf=3, min_samples_split=2, n_estimators=800; total time=   1.7s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=3, min_samples_split=2, n_estimators=900; total time=   1.8s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=3, min_samples_split=2, n_estimators=900; total time=   1.9s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=3, min_samples_split=2, n_estimators=900; total time=   1.8s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=3, min_samples_split=2, n_estimators=900; total time=   1.9s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=3, min_samples_split=2, n_estimators=900; total time=   1.8s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=3, min_samples_split=2, n_estimators=1000; total time=   2.0s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=3, min_samples_split=2, n_estimators=1000; total time=   2.0s
[CV] END bootstrap=False, max_depth=30, min_samples_le

[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=1000; total time=   1.9s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=3, n_estimators=700; total time=   1.3s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=3, n_estimators=700; total time=   1.3s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=3, n_estimators=700; total time=   1.3s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=3, n_estimators=700; total time=   1.3s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=3, n_estimators=700; total time=   1.3s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=3, n_estimators=800; total time=   1.5s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=4, min_samples_split=3, n_estimators=800; total time=   1.5s
[CV] END bootstrap=False, max_depth=30, min_samples_lea

[CV] END bootstrap=False, max_depth=40, min_samples_leaf=2, min_samples_split=3, n_estimators=800; total time=   1.8s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=2, min_samples_split=3, n_estimators=900; total time=   2.1s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=2, min_samples_split=3, n_estimators=900; total time=   2.1s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=2, min_samples_split=3, n_estimators=900; total time=   2.1s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=2, min_samples_split=3, n_estimators=900; total time=   2.1s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=2, min_samples_split=3, n_estimators=900; total time=   2.1s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=2, min_samples_split=3, n_estimators=1000; total time=   2.3s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=2, min_samples_split=3, n_estimators=1000; total time=   2.3s
[CV] END bootstrap=False, max_depth=40, min_samples_le

[CV] END bootstrap=False, max_depth=40, min_samples_leaf=3, min_samples_split=3, n_estimators=1000; total time=   2.1s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=3, min_samples_split=4, n_estimators=700; total time=   1.4s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=3, min_samples_split=4, n_estimators=700; total time=   1.4s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=3, min_samples_split=4, n_estimators=700; total time=   1.4s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=3, min_samples_split=4, n_estimators=700; total time=   1.4s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=3, min_samples_split=4, n_estimators=700; total time=   1.4s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=3, min_samples_split=4, n_estimators=800; total time=   1.6s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=3, min_samples_split=4, n_estimators=800; total time=   1.6s
[CV] END bootstrap=False, max_depth=40, min_samples_lea

[CV] END bootstrap=False, max_depth=40, min_samples_leaf=4, min_samples_split=4, n_estimators=800; total time=   1.5s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=4, min_samples_split=4, n_estimators=900; total time=   1.7s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=4, min_samples_split=4, n_estimators=900; total time=   1.7s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=4, min_samples_split=4, n_estimators=900; total time=   1.7s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=4, min_samples_split=4, n_estimators=900; total time=   1.7s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=4, min_samples_split=4, n_estimators=900; total time=   1.7s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=4, min_samples_split=4, n_estimators=1000; total time=   1.9s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=4, min_samples_split=4, n_estimators=1000; total time=   1.9s
[CV] END bootstrap=False, max_depth=40, min_samples_le

[CV] END bootstrap=False, max_depth=50, min_samples_leaf=2, min_samples_split=4, n_estimators=1000; total time=   2.3s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=3, min_samples_split=2, n_estimators=700; total time=   1.4s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=3, min_samples_split=2, n_estimators=700; total time=   1.4s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=3, min_samples_split=2, n_estimators=700; total time=   1.4s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=3, min_samples_split=2, n_estimators=700; total time=   1.4s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=3, min_samples_split=2, n_estimators=700; total time=   1.4s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=3, min_samples_split=2, n_estimators=800; total time=   1.6s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=3, min_samples_split=2, n_estimators=800; total time=   1.6s
[CV] END bootstrap=False, max_depth=50, min_samples_lea

[CV] END bootstrap=False, max_depth=50, min_samples_leaf=4, min_samples_split=2, n_estimators=800; total time=   1.5s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=4, min_samples_split=2, n_estimators=900; total time=   1.7s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=4, min_samples_split=2, n_estimators=900; total time=   1.7s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=4, min_samples_split=2, n_estimators=900; total time=   1.7s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=4, min_samples_split=2, n_estimators=900; total time=   1.7s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=4, min_samples_split=2, n_estimators=900; total time=   1.7s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=4, min_samples_split=2, n_estimators=1000; total time=   1.9s
[CV] END bootstrap=False, max_depth=50, min_samples_leaf=4, min_samples_split=2, n_estimators=1000; total time=   1.9s
[CV] END bootstrap=False, max_depth=50, min_samples_le

GridSearchCV(cv=5,
             estimator=RandomForestRegressor(max_features='sqrt',
                                             random_state=42),
             param_grid={'bootstrap': [False], 'max_depth': [30, 40, 50],
                         'min_samples_leaf': [2, 3, 4],
                         'min_samples_split': [2, 3, 4],
                         'n_estimators': [700, 800, 900, 1000]},
             verbose=2)

In [45]:
grid.best_params_

{'bootstrap': False,
 'max_depth': 30,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 800}

In [61]:
print("RF train R_squared: %0.3f" % grid.best_score_)

RF train R_squared: 0.388


In [62]:
print("RF test R_squared: %0.3f" % grid.score(X_test, y_test))

RF test R_squared: 0.398


In [52]:
def model_eval_metrics(y_true, y_pred,classification="TRUE"):
     if classification=="TRUE":
        accuracy_eval = accuracy_score(y_true, y_pred)
        f1_score_eval = f1_score(y_true, y_pred,average="macro",zero_division=0)
        precision_eval = precision_score(y_true, y_pred,average="macro",zero_division=0)
        recall_eval = recall_score(y_true, y_pred,average="macro",zero_division=0)
        mse_eval = 0
        rmse_eval = 0
        mae_eval = 0
        r2_eval = 0
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     else:
        accuracy_eval = 0
        f1_score_eval = 0
        precision_eval = 0
        recall_eval = 0
        mse_eval = mean_squared_error(y_true, y_pred)
        rmse_eval = sqrt(mean_squared_error(y_true, y_pred))
        mae_eval = mean_absolute_error(y_true, y_pred)
        r2_eval = r2_score(y_true, y_pred)
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     return finalmetricdata

In [54]:
final_rf = RandomForestRegressor(bootstrap = False,
 max_depth = 30,
 min_samples_leaf = 2,
 min_samples_split = 2,
 n_estimators = 800, max_features = 'sqrt', random_state=42)

In [56]:
final_rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=False, max_depth=30, max_features='sqrt',
                      min_samples_leaf=2, n_estimators=800, random_state=42)

In [66]:
y_pred = final_rf.predict(X_test)

In [60]:
model_eval_metrics(y_test, y_pred, classification = False)

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0,0,0,0,1.086113,1.042167,0.818344,0.398287


## Vs Base Model (For Comparison)

In [63]:
base_rf = RandomForestRegressor(random_state=42)

In [64]:
base_rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [67]:
y_pred_base = base_rf.predict(X_test)

In [68]:
model_eval_metrics(y_test, y_pred_base, classification = False)

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0,0,0,0,1.116146,1.056478,0.824189,0.381648


Indeed RMSE has fallen, and r^2 is slightly better for `final_rf`, relative to the untuned `base_rf`.

# Permutation Importance without Clustering

https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-py

Why permutation importances and not feature importances? Because impurity-based feature importances are subject to cardinality of features, and favour high cardinality features (e.g., numerical features with large scales). Feature importances are also usually calculated only on a single set of data, and cannot be evaluated on a held-out validation set, to check for the generalisability of these permutation importances. 

Permutation-based importances overcome these problems.

**How to Interpet**:
The values here are the change in R^2 or RMSE for a given model, should this feature be removed. 

## On Training Set

### R-Squared

In [89]:
from sklearn.inspection import permutation_importance
r1 = permutation_importance(final_rf, X_train, y_train, n_repeats=30,random_state=42)

In [90]:
for i in r1.importances_mean.argsort()[::-1]:
    if r1.importances_mean[i] - 2 * r1.importances_std[i] > 0:
        print(f"{X_train.columns.values[i]:<8}" " "
              f"{r1.importances_mean[i]:.3f}"
              f" +/- {r1.importances_std[i]:.3f}")

school_size 0.274 +/- 0.007
school_type 0.187 +/- 0.006
open_house 0.125 +/- 0.003
percentacadnotimptlog1p 0.111 +/- 0.003
crime_live_recode 0.100 +/- 0.003
parent_teacher_conf 0.097 +/- 0.003
percentlowgradeslog1p 0.090 +/- 0.002
juvhelp_drug 0.075 +/- 0.003
random_sweep 0.064 +/- 0.003
crime_area_recode 0.058 +/- 0.003
anonymous_report 0.057 +/- 0.003
mhhelp_drug 0.047 +/- 0.002
school_lockers 0.042 +/- 0.002
prohibit_phone 0.039 +/- 0.002
dress_code 0.037 +/- 0.001
socialshelp_drug 0.032 +/- 0.001
urban    0.030 +/- 0.001
control_ground 0.026 +/- 0.001
relighelp_drug 0.026 +/- 0.001
civichelp_drug 0.023 +/- 0.001
close_lunch 0.023 +/- 0.001
student_id 0.022 +/- 0.001
restorative_circles 0.021 +/- 0.001
disciplinary_process 0.021 +/- 0.001
staff_id 0.021 +/- 0.001
privatehelp_drug 0.021 +/- 0.001
parent_training_assist 0.020 +/- 0.001
lock_inside 0.020 +/- 0.001
emergency_notif 0.019 +/- 0.001
peer_mediation 0.019 +/- 0.001
parenthelp_drug 0.018 +/- 0.001
panic_button 0.018 +/- 0.001

### RMSE

In [99]:
from sklearn.inspection import permutation_importance
r1_1 = permutation_importance(final_rf, X_train, y_train, scoring = 'neg_root_mean_squared_error', n_repeats=30,random_state=42)

In [100]:
for i in r1_1.importances_mean.argsort()[::-1]:
    if r1_1.importances_mean[i] - 2 * r1_1.importances_std[i] > 0:
        print(f"{X_train.columns.values[i]:<8}" " "
              f"{r1_1.importances_mean[i]:.3f}"
              f" +/- {r1_1.importances_std[i]:.3f}")

school_size 0.414 +/- 0.008
school_type 0.309 +/- 0.008
open_house 0.224 +/- 0.004
percentacadnotimptlog1p 0.203 +/- 0.004
crime_live_recode 0.186 +/- 0.005
parent_teacher_conf 0.181 +/- 0.005
percentlowgradeslog1p 0.170 +/- 0.004
juvhelp_drug 0.146 +/- 0.004
random_sweep 0.127 +/- 0.004
crime_area_recode 0.116 +/- 0.005
anonymous_report 0.114 +/- 0.005
mhhelp_drug 0.095 +/- 0.003
school_lockers 0.087 +/- 0.003
prohibit_phone 0.082 +/- 0.003
dress_code 0.077 +/- 0.003
socialshelp_drug 0.068 +/- 0.003
urban    0.065 +/- 0.002
control_ground 0.056 +/- 0.002
relighelp_drug 0.055 +/- 0.002
civichelp_drug 0.050 +/- 0.002
close_lunch 0.049 +/- 0.002
student_id 0.048 +/- 0.002
restorative_circles 0.046 +/- 0.002
disciplinary_process 0.045 +/- 0.002
staff_id 0.045 +/- 0.002
privatehelp_drug 0.045 +/- 0.002
parent_training_assist 0.043 +/- 0.002
lock_inside 0.043 +/- 0.002
emergency_notif 0.042 +/- 0.002
peer_mediation 0.041 +/- 0.002
parenthelp_drug 0.040 +/- 0.001
panic_button 0.040 +/- 0.001

## On Test Set

### R-Squared

In [92]:
from sklearn.inspection import permutation_importance
r2 = permutation_importance(final_rf, X_test, y_test, n_repeats=30,random_state=42)

In [76]:
X_test

,percentlowgradeslog1p,percentacadnotimptlog1p,school_size,urban,crime_live_recode,crime_area_recode,school_type,visitor_badge,control_building,control_ground,...,privatehelp_drug,relighelp_drug,prevention_curriculum,sel,behav_mod,mentoring,peer_mediation,student_court,restorative_circles,promote_community
367,1.791759,1.791759,4,1,1,1,1,1,1,1,...,0,1,0,0,1,1,1,0,0,1
2760,2.397895,2.397895,3,1,2,1,1,1,1,0,...,0,0,1,1,1,1,1,0,1,1
1330,2.397895,3.433987,3,0,1,1,2,1,1,0,...,0,0,1,1,1,1,1,1,1,1
2136,4.394449,0.000000,3,1,2,2,2,1,1,0,...,0,0,1,1,1,1,1,0,0,1
521,2.397895,3.044522,2,1,2,2,0,1,1,1,...,0,0,1,1,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,2.397895,2.397895,2,1,2,3,2,1,1,1,...,0,0,1,1,1,1,0,0,1,1
1724,2.772589,3.258097,4,1,1,1,1,1,1,0,...,1,1,0,0,1,1,1,0,0,1
764,2.772589,4.330733,1,0,1,1,2,1,1,0,...,0,0,1,1,1,1,0,0,0,0
1684,2.397895,2.772589,4,1,2,1,1,1,1,1,...,1,0,1,1,1,1,1,0,1,1


In [93]:
for i in r2.importances_mean.argsort()[::-1]:
    if r2.importances_mean[i] - 2 * r2.importances_std[i] > 0:
        print(f"{X_test.columns.values[i]:<8}" " "
              f"{r2.importances_mean[i]:.3f}"
              f" +/- {r2.importances_std[i]:.3f}")

school_size 0.177 +/- 0.015
school_type 0.092 +/- 0.011
open_house 0.021 +/- 0.006
percentlowgradeslog1p 0.017 +/- 0.003
percentacadnotimptlog1p 0.015 +/- 0.006
parent_teacher_conf 0.014 +/- 0.005
random_sweep 0.010 +/- 0.003
school_lockers 0.006 +/- 0.002
staff_id 0.003 +/- 0.001


### RMSE

In [101]:
from sklearn.inspection import permutation_importance
r2_1 = permutation_importance(final_rf, X_test, y_test, scoring = 'neg_root_mean_squared_error', n_repeats=30,random_state=42)

In [102]:
for i in r2_1.importances_mean.argsort()[::-1]:
    if r2_1.importances_mean[i] - 2 * r2_1.importances_std[i] > 0:
        print(f"{X_test.columns.values[i]:<8}" " "
              f"{r2_1.importances_mean[i]:.3f}"
              f" +/- {r2_1.importances_std[i]:.3f}")

school_size 0.143 +/- 0.012
school_type 0.077 +/- 0.009
open_house 0.018 +/- 0.005
percentlowgradeslog1p 0.014 +/- 0.003
percentacadnotimptlog1p 0.013 +/- 0.005
parent_teacher_conf 0.012 +/- 0.004
random_sweep 0.008 +/- 0.003
school_lockers 0.005 +/- 0.002
staff_id 0.002 +/- 0.001
